<a href="https://colab.research.google.com/github/vybhav2000/GRIP-SparksFoundation-Tasks/blob/master/task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
plt.rcParams['figure.figsize']=12,7

### Importing Data

In [2]:
data=pd.read_csv('Iris.csv')

In [3]:
data.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
len(data)

150

In [5]:
data.columns

Index(['Id', 'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm',
       'Species'],
      dtype='object')

In [6]:
cols=['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']

### Doing some visualisation

In [7]:
fig=px.bar(data,x='Species',color='Species')
fig.show()

In [8]:
fig=px.scatter_matrix(data,data[cols],color='Species')
fig.update_layout(width=1000,height=1000)
fig.show()

### Finding the optimum number of clusters

In [9]:
dat=data[cols].values
sse=[]
for k in range(1,11):
    means=KMeans(n_clusters=k).fit(dat)
    sse.append(means.inertia_)

In [10]:
sse

[680.8244,
 152.36870647733906,
 78.94084142614602,
 57.31787321428571,
 46.53558205128205,
 38.953730364980366,
 34.299985543844244,
 29.87991975437056,
 27.839591136933244,
 26.424343434343438]

In [11]:
fig=px.line(x=[x for x in range(1,11)],y=sse,title='WSSE vs Number of clusters')
fig.add_traces(px.scatter(x=[x for x in range(1,11)],y=sse).data[0])
fig.update_traces(marker=dict(size=12,line=dict(width=2,color='darkred')),selector=dict(mode='markers'))
fig.update_xaxes(tickvals=[x for x in range(11)],title='Number of clusters')
fig.update_yaxes(title='Within Sum of Squares Error')
fig.show()

In [12]:
print('The optimum number of clusters as identified from the elbow method is 3')

The optimum number of clusters as identified from the elbow method is 3


### Fitting the data and making predictions

In [13]:
means=KMeans(n_clusters=3).fit(dat)
preds=means.predict(dat)

In [14]:
mapper={1:'Iris-setosa',0:'Iris-versicolor',2:'Iris-virginica'}

In [15]:
data['Mapped_pred']=preds

In [16]:
data['Mapped_pred']=data['Mapped_pred'].map(mapper)

In [17]:
data['Mapped_pred']

0          Iris-setosa
1          Iris-setosa
2          Iris-setosa
3          Iris-setosa
4          Iris-setosa
            ...       
145     Iris-virginica
146    Iris-versicolor
147     Iris-virginica
148     Iris-virginica
149    Iris-versicolor
Name: Mapped_pred, Length: 150, dtype: object

### Plotting the predictions

In [18]:
fig=px.scatter_matrix(data,data[cols],color='Mapped_pred')
fig.update_layout(width=1000,height=1000)
fig.show()

In [19]:
from sklearn.metrics import classification_report,confusion_matrix

In [20]:
print(classification_report(data['Species'],data['Mapped_pred']))

                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        50
Iris-versicolor       0.77      0.96      0.86        50
 Iris-virginica       0.95      0.72      0.82        50

       accuracy                           0.89       150
      macro avg       0.91      0.89      0.89       150
   weighted avg       0.91      0.89      0.89       150



In [21]:
cm=confusion_matrix(data['Species'],data['Mapped_pred'])

In [22]:
cm

array([[50,  0,  0],
       [ 0, 48,  2],
       [ 0, 14, 36]])

### There seems to be a few misclassifications between versiclor and virginica 

In [23]:
fig=ff.create_annotated_heatmap(cm,x=['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'],y=['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'])
fig.update_xaxes(title='Predicted')
fig.update_yaxes(title='True')
fig.show()